### 데이터 전처리

In [7]:
import pandas as pd
import numpy as np
import wfdb
import ast

def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = "./"
sampling_rate=100

Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

X = load_raw_data(Y, sampling_rate, path)

agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer
#이 부분이 실행이 안 되시면 prompt에서 sklearn.preprocessing 설치하시면 됩니다
Y = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

agg_df = pd.read_csv(path + 'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

empty_indices = Y[Y['diagnostic_superclass'].apply(lambda x: len(x) == 0)].index
Y = Y.drop(empty_indices)

X = load_raw_data(Y, sampling_rate, path)  

test_fold = 10
validation_fold = 9

X_test = X[Y.strat_fold == test_fold]
y_test = Y[Y.strat_fold == test_fold]['diagnostic_superclass']

X_validation = X[Y.strat_fold == validation_fold]
y_validation = Y[Y.strat_fold == validation_fold]['diagnostic_superclass']

X_train = X[(Y.strat_fold != test_fold) & (Y.strat_fold != validation_fold)]
y_train = Y[(Y.strat_fold != test_fold) & (Y.strat_fold != validation_fold)]['diagnostic_superclass']

mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train)
y_test_bin = mlb.transform(y_test)
y_validation_bin = mlb.transform(y_validation)


### Data Transform

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import ViTForImageClassification, DeiTForImageClassification
from torchvision.models import convnext_tiny, efficientnet_v2_s
import numpy as np
import matplotlib.pyplot as plt
import os

# Custom Dataset for ECG Data
class ECGDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.expand(3, -1, -1)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

dataset = ECGDataset(X_train, y_train_bin, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)



In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### ViT

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torchvision import transforms
from transformers import ViTForImageClassification
from torch.utils.data import DataLoader

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device} (GPU 1)")

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
num_classes = y_train_bin.shape[1]
model.classifier = nn.Linear(model.classifier.in_features, num_classes)
model = model.to(device)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.expand(3, -1, -1)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def train_model(model, dataloader, num_epochs=40, patience=3):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    best_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print(f"Starting epoch {epoch+1}/{num_epochs}...")
        running_loss = 0.0
        all_labels = []
        all_preds = []
        for batch_idx, (inputs, labels) in enumerate(dataloader):
            inputs, labels = inputs.to(device), labels.float().to(device)
            optimizer.zero_grad()
            outputs = model(inputs).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            preds = (torch.sigmoid(outputs) > 0.5).int().cpu().numpy()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        accuracy = accuracy_score(np.vstack(all_labels), np.vstack(all_preds))
        f1 = f1_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        precision = precision_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        recall = recall_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

print('Training ViT...')
train_model(model, dataloader)
print('ViT training completed.')


Using device: cuda:1 (GPU 1)
Training ViT...
Starting epoch 1/40...
Epoch [1/40], Loss: 0.5519, Accuracy: 0.0524, F1 Score: 0.0422, Precision: 0.2565, Recall: 0.0269
Starting epoch 2/40...
Epoch [2/40], Loss: 0.5237, Accuracy: 0.2197, F1 Score: 0.1354, Precision: 0.4102, Recall: 0.1166
Starting epoch 3/40...
Epoch [3/40], Loss: 0.4837, Accuracy: 0.3269, F1 Score: 0.2515, Precision: 0.5325, Recall: 0.2132
Starting epoch 4/40...
Epoch [4/40], Loss: 0.4598, Accuracy: 0.3629, F1 Score: 0.3275, Precision: 0.5767, Recall: 0.2754
Starting epoch 5/40...
Epoch [5/40], Loss: 0.4452, Accuracy: 0.3832, F1 Score: 0.4025, Precision: 0.6309, Recall: 0.3336
Starting epoch 6/40...
Epoch [6/40], Loss: 0.4345, Accuracy: 0.3976, F1 Score: 0.4385, Precision: 0.6417, Recall: 0.3646
Starting epoch 7/40...
Epoch [7/40], Loss: 0.4285, Accuracy: 0.4086, F1 Score: 0.4536, Precision: 0.6495, Recall: 0.3788
Starting epoch 8/40...
Epoch [8/40], Loss: 0.4244, Accuracy: 0.4090, F1 Score: 0.4596, Precision: 0.6431, Re

### DeiT

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torchvision import transforms
from torchvision.models import convnext_tiny
from torch.utils.data import DataLoader
from transformers import DeiTForImageClassification


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = convnext_tiny(pretrained=True)
num_classes = y_train_bin.shape[1]  
model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes)
model = model.to(device)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),  
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

def train_model(model, dataloader, num_epochs=40, patience=3):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    best_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print(f"Starting epoch {epoch+1}/{num_epochs}...")
        running_loss = 0.0
        all_labels = []
        all_preds = []
        for batch_idx, (inputs, labels) in enumerate(dataloader):
            inputs, labels = inputs.to(device), labels.float().to(device)
            optimizer.zero_grad()
            outputs = model(inputs).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            preds = (torch.sigmoid(outputs) > 0.5).int().cpu().numpy()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        accuracy = accuracy_score(np.vstack(all_labels), np.vstack(all_preds))
        f1 = f1_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        precision = precision_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        recall = recall_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

model = DeiTForImageClassification.from_pretrained('facebook/deit-base-distilled-patch16-224')
num_classes = y_train_bin.shape[1]
model.classifier = nn.Linear(model.classifier.in_features, num_classes)
model = model.to(device)

print('Training DeiT...')
train_model(model, dataloader)
print('DeiT training completed.')


Using device: cuda


/home/gaya/anaconda3/envs/dissc/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gaya/anaconda3/envs/dissc/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training DeiT...
Starting epoch 1/40...
Epoch [1/40], Loss: 0.5563, Accuracy: 0.0789, F1 Score: 0.0568, Precision: 0.2698, Recall: 0.0392
Starting epoch 2/40...
Epoch [2/40], Loss: 0.5272, Accuracy: 0.1911, F1 Score: 0.1450, Precision: 0.4413, Recall: 0.1126
Starting epoch 3/40...
Epoch [3/40], Loss: 0.4997, Accuracy: 0.2978, F1 Score: 0.2279, Precision: 0.5108, Recall: 0.1916
Starting epoch 4/40...
Epoch [4/40], Loss: 0.4939, Accuracy: 0.3142, F1 Score: 0.2607, Precision: 0.5483, Recall: 0.2151
Starting epoch 5/40...
Epoch [5/40], Loss: 0.4925, Accuracy: 0.3077, F1 Score: 0.2615, Precision: 0.5485, Recall: 0.2134
Starting epoch 6/40...
Epoch [6/40], Loss: 0.4849, Accuracy: 0.3213, F1 Score: 0.2975, Precision: 0.5729, Recall: 0.2394
Starting epoch 7/40...
Epoch [7/40], Loss: 0.4884, Accuracy: 0.3101, F1 Score: 0.2724, Precision: 0.5583, Recall: 0.2200
Starting epoch 8/40...
Epoch [8/40], Loss: 0.4811, Accuracy: 0.3277, F1 Score: 0.2879, Precision: 0.5609, Recall: 0.2356
Starting epoch 

### EfficientV2

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torchvision import transforms
from torchvision.models import efficientnet_v2_s
from torch.utils.data import DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


model = efficientnet_v2_s(pretrained=True)
num_classes = y_train_bin.shape[1]  
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)), 
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

def train_model(model, dataloader, num_epochs=40, patience=3):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    best_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print(f"Starting epoch {epoch+1}/{num_epochs}...")
        running_loss = 0.0
        all_labels = []
        all_preds = []
        for batch_idx, (inputs, labels) in enumerate(dataloader):
            inputs, labels = inputs.to(device).float(), labels.to(device).float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            preds = (torch.sigmoid(outputs) > 0.5).int().cpu().numpy()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        accuracy = accuracy_score(np.vstack(all_labels), np.vstack(all_preds))
        f1 = f1_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        precision = precision_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        recall = recall_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')


        if epoch_loss < best_loss:
            best_loss = epoch_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break


print('Training EfficientNetV2...')
train_model(model, dataloader)
print('EfficientNetV2 training completed.')


Using device: cuda


/home/gaya/anaconda3/envs/dissc/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gaya/anaconda3/envs/dissc/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training EfficientNetV2...
Starting epoch 1/40...
Epoch [1/40], Loss: 0.3724, Accuracy: 0.5009, F1 Score: 0.5880, Precision: 0.7039, Recall: 0.5232
Starting epoch 2/40...
Epoch [2/40], Loss: 0.3209, Accuracy: 0.5655, F1 Score: 0.6690, Precision: 0.7576, Recall: 0.6109
Starting epoch 3/40...
Epoch [3/40], Loss: 0.3031, Accuracy: 0.5897, F1 Score: 0.6907, Precision: 0.7755, Recall: 0.6337
Starting epoch 4/40...
Epoch [4/40], Loss: 0.2913, Accuracy: 0.5956, F1 Score: 0.7025, Precision: 0.7793, Recall: 0.6487
Starting epoch 5/40...
Epoch [5/40], Loss: 0.2807, Accuracy: 0.6075, F1 Score: 0.7147, Precision: 0.7885, Recall: 0.6626
Starting epoch 6/40...
Epoch [6/40], Loss: 0.2721, Accuracy: 0.6220, F1 Score: 0.7260, Precision: 0.7976, Recall: 0.6750
Starting epoch 7/40...
Epoch [7/40], Loss: 0.2661, Accuracy: 0.6277, F1 Score: 0.7342, Precision: 0.8020, Recall: 0.6851
Starting epoch 8/40...
Epoch [8/40], Loss: 0.2562, Accuracy: 0.6408, F1 Score: 0.7437, Precision: 0.8104, Recall: 0.6952
Start

### ConVNext

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torchvision import transforms
from torchvision.models import convnext_tiny
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = convnext_tiny(pretrained=True)
num_classes = y_train_bin.shape[1] 
model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes)
model = model.to(device)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)), 
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)), 
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def train_model(model, dataloader, num_epochs=40, patience=3):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    best_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print(f"Starting epoch {epoch+1}/{num_epochs}...")
        running_loss = 0.0
        all_labels = []
        all_preds = []
        for batch_idx, (inputs, labels) in enumerate(dataloader):
            inputs, labels = inputs.to(device).float(), labels.to(device).float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            preds = (torch.sigmoid(outputs) > 0.5).int().cpu().numpy()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        accuracy = accuracy_score(np.vstack(all_labels), np.vstack(all_preds))
        f1 = f1_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        precision = precision_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        recall = recall_score(np.vstack(all_labels), np.vstack(all_preds), average='macro')
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

print('Training ConvNeXt...')
train_model(model, dataloader)
print('ConvNeXt training completed.')


Using device: cuda


/home/gaya/anaconda3/envs/dissc/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gaya/anaconda3/envs/dissc/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training ConvNeXt...
Starting epoch 1/40...
Epoch [1/40], Loss: 0.5492, Accuracy: 0.0328, F1 Score: 0.0282, Precision: 0.2657, Recall: 0.0165
Starting epoch 2/40...
Epoch [2/40], Loss: 0.5468, Accuracy: 0.0163, F1 Score: 0.0153, Precision: 0.2415, Recall: 0.0083
Starting epoch 3/40...
Epoch [3/40], Loss: 0.5464, Accuracy: 0.0023, F1 Score: 0.0023, Precision: 0.1444, Recall: 0.0012
Starting epoch 4/40...


/home/gaya/anaconda3/envs/dissc/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [4/40], Loss: 0.5459, Accuracy: 0.0039, F1 Score: 0.0036, Precision: 0.0897, Recall: 0.0018
Starting epoch 5/40...


...

/home/gaya/anaconda3/envs/dissc/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [19/40], Loss: 0.5456, Accuracy: 0.0001, F1 Score: 0.0001, Precision: 0.1000, Recall: 0.0001
Starting epoch 20/40...


/home/gaya/anaconda3/envs/dissc/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [20/40], Loss: 0.5455, Accuracy: 0.0025, F1 Score: 0.0024, Precision: 0.2193, Recall: 0.0012
Starting epoch 21/40...
Epoch [21/40], Loss: 0.5456, Accuracy: 0.0001, F1 Score: 0.0001, Precision: 0.0500, Recall: 0.0000
Early stopping triggered after 21 epochs.
ConvNeXt training completed.


/home/gaya/anaconda3/envs/dissc/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
